### Transform orders data - string to JSON 
1. Pre-process the JSON string to fix the dat quality issues
2. Transform JSON string to JSON obj
3. Write transformed data to silver schema


In [0]:
select * from gizmobox.bronze.v_orders

### Pre-process the JSON string to fix the dat quality issues

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_fixed_value AS
SELECT
  regexp_replace(
    value,
    '"order_date": (\\d{4}-\\d{2}-\\d{2})(?!")',
    '"order_date": "' || regexp_extract(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})(?!")', 1) || '"'
  ) AS fixed_value
FROM gizmobox.bronze.v_orders;



### Transform JSON string to JSON obj

In [0]:
select schema_of_json(fixed_value) from tv_fixed_value
limit 1;

In [0]:
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as JSON_value, fixed_value
from tv_fixed_value

### Write transformed data to silver schema

In [0]:
DROP TABLE IF EXISTS gizmobox.silver.orders;
CREATE TABLE  gizmobox.silver.orders_json AS
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as JSON_value
from tv_fixed_value

In [0]:
select * from gizmobox.silver.orders_json

### Transform Orders Data - Explode Arrays
1. Access elements from JSON object.
2. Deduplicate Array elements.
3. Explode Arrays
4. Write the transformed data to silver schema

### Access elements from JSON object.

In [0]:
select JSON_value.customer_id,
JSON_value.order_date,
JSON_value.order_id,
JSON_value.order_status, 
JSON_value.payment_method,
JSON_value.total_amount,
JSON_value.transaction_timestamp,
JSON_value.items
from gizmobox.silver.orders_json 

### Deduplicate Array elements.
array_distinct(arr)

In [0]:
select JSON_value.customer_id,
JSON_value.order_date,
JSON_value.order_id,
JSON_value.order_status, 
JSON_value.payment_method,
JSON_value.total_amount,
JSON_value.transaction_timestamp,
array_distinct(JSON_value.items) as items
from gizmobox.silver.orders_json 

### Explode Arrays
explode(arr)

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_explode AS
select JSON_value.customer_id,
JSON_value.order_date,
JSON_value.order_id,
JSON_value.order_status, 
JSON_value.payment_method,
JSON_value.total_amount,
JSON_value.transaction_timestamp,
explode(array_distinct(JSON_value.items)) as item
from gizmobox.silver.orders_json 

In [0]:
select customer_id,
order_date,
order_id,
order_status, 
payment_method,
total_amount,
transaction_timestamp,
 item.item_id,
 item.name,
 item.price,
 item.quantity,
 item.category,
 item.details.brand,
 item.details.color
from tv_orders_explode

Write the transformed data to silver schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders AS
select customer_id,
order_date,
order_id,
order_status, 
payment_method,
total_amount,
transaction_timestamp,
 item.item_id,
 item.name,
 item.price,
 item.quantity,
 item.category,
 item.details.brand,
 item.details.color
from tv_orders_explode

In [0]:
select * from gizmobox.silver.orders